In [1]:
import os
import pandas as pd
from pathlib import Path
import seaborn as sns
import matplotlib.pyplot as plt

os.chdir(os.getcwd() / Path('GRETA_2021_cf'))

In [2]:
# Reading current cf data

coordinate_df = pd.read_excel("coordinate.xlsx")
coordinate_df.dropna(inplace=True)
coordinate_df.columns = ['lon', 'lat', 'province', 'merra_lon', 'merra_lat', 'grid cell']

windcf = pd.read_csv("windcf.csv",header=None)
windcf.columns = ['hour', 'grid cell', 'cf']

distance = pd.read_csv("distance_to_grid.csv",header=None)
distance.columns = ['grid cell', 'distance']

In [3]:
windcf_prov = windcf.merge(coordinate_df[['grid cell', 'province']], on='grid cell', how='left')
windcf_prov.dropna(subset='province',inplace=True)

In [4]:
provinces = windcf_prov['province'].unique()
windcf_province = {}

for province in provinces:
    df = windcf_prov.loc[windcf_prov['province'] == province]
    df = pd.pivot_table(df, values='cf', index=['hour'], columns=['grid cell'])
    df.loc['total'] = df.iloc[0:8758].sum()
    df.loc['mean'] = df.iloc[0:8758].mean()
    df.loc['std'] = df.iloc[0:8758].std()
    df.loc['median'] = df.iloc[0:8758].median()
    df.to_csv(f'windcf_{province}.csv')
    windcf_province[province] = df

In [5]:
for province in provinces:
    df = pd.DataFrame(windcf_province[province])
    df = df.iloc[0:8758]
    df = df.to_numpy().flatten()
    df = df[df != 0]
    plt.hist(df, color='green', bins=20)
    plt.title(f'Average VRE Capacity Factor in {province}')
    plt.xlabel('Capacity Factor')
    plt.ylabel('Frequency')
    plt.savefig(f'cf_hist_{province}.png', dpi=300, bbox_inches='tight')
    plt.close()
    del df

In [12]:
for province in provinces:
    df = pd.DataFrame(windcf_province[province])
    df = pd.DataFrame(df.loc['mean'].T)
    df = df.merge(distance, on='grid cell', how='left')
    sns.lmplot(x='distance', y='mean', data=df)
    plt.title(f'Wind Capacity Factor vs Distance to Grid in {province}')
    plt.xlabel('Distance to Grid (km)')
    plt.ylabel('Annual average Capacity Factor')
    plt.savefig(f'windcf_distance_{province}.jpg', dpi=300, bbox_inches='tight')
    plt.close()
    del df